<a href="https://colab.research.google.com/github/Sparrow0hawk/police-api-client-python/blob/develop/notebooks/police_api_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing out the police api python package

Having a go at extracting crime data from the Python Police API package and getting outputs that look like the bulk data extraction available on [Custom Download section](https://data.police.uk/data/).

In [ ]:
! pip install police-api-client

## Example data

In [ ]:
# an example of the target data structure we're aiming for 
import pandas as pd

print(pd.read_csv('https://raw.githubusercontent.com/Sparrow0hawk/crime_sim_toolkit/master/sample_data/2019-06/2019-06-west-yorkshire-street.csv').shape)

pd.read_csv('https://raw.githubusercontent.com/Sparrow0hawk/crime_sim_toolkit/master/sample_data/2019-06/2019-06-west-yorkshire-street.csv')\
  .head()

## Exploring the API

In [ ]:
# standard instanstiation
from police_api import PoliceAPI
api = PoliceAPI()

In [ ]:
# available dates from API
api.get_dates()

In [ ]:
# available police forces 
# returns a list of Force objects which have their own attributes
api.get_forces()

In [ ]:
# Forces object attributes
api.get_forces()[0].__dict__

In [ ]:
print(api.get_forces()[0].description, '\n')

print(api.get_forces()[0].telephone, '\n')

print(api.get_forces()[0].engagement_methods, '\n')

print(api.get_forces()[0].url, '\n')

In [ ]:
# assigning specific force object for requested force
WY = api.get_force('west-yorkshire')

In [ ]:
WY.fields

In [ ]:
type(WY)

In [ ]:
# get neighbourhoods

In [ ]:
WY.neighbourhoods

In [ ]:
WY.neighbourhoods[0].__dict__

In [ ]:
lds_cc = WY.get_neighbourhood('LDT_CITY')

In [ ]:
lds_cc.crimes

In [ ]:
crimes = api.get_crimes_area(lds_cc.boundary, date='2020-03')

In [ ]:
crimes[0].__dict__

In [ ]:
type(crimes[0])

In [ ]:
crimes[0].__dict__.keys()

In [ ]:
WY.name

In [ ]:
list(crimes[0].__dict__.values())

In [ ]:
crime_dict = { key: [] for key in list(crimes[0].__dict__.keys())}

In [ ]:
crimes[0].__dict__['api']

In [ ]:
empty_dic = dict()

for crime in crimes:

  new_dict = { **empty_dic, **crime.__dict__}

## Iterating over crime objects

In [ ]:
crime_dict = { key: [] for key in list(crimes[0].__dict__.keys())}

crime_dict['lon'] = []
crime_dict['lat'] = []
crime_dict['crime_cat'] = []

for crime in crimes:

  for key in list(crimes[0].__dict__.keys()):

    crime_dict[key] += [crime.__dict__[key]]

  # extract location lng and lat
  crime_dict['lon'] += [crime.__dict__['location'].__dict__['longitude']]

  crime_dict['lat'] += [crime.__dict__['location'].__dict__['latitude']]
  # extract crime category

  crime_dict['crime_cat'] += [crime.__dict__['category'].__dict__['name']]

In [ ]:
import pandas as pd

data = pd.DataFrame(crime_dict)

data.head()

In [ ]:
from police_api import PoliceAPI
import pandas as pd

def policeAPI_to_pandas(police_force : str, date : str):
  """
  Using the policeAPI to get data into pandas dataframe
  """
  api = PoliceAPI()

  pile_of_dataframes = []

  # assigning specific force object for requested force
  selected_force = api.get_force('west-yorkshire')

  # get list of neighbourhoods
  force_neighbourhoods = [obj.id for obj in selected_force.neighbourhoods]

  for neighbourhood in force_neighbourhoods:

    crimes = api.get_crimes_area(selected_force.get_neighbourhood(neighbourhood).boundary, 
                               date=date)
    
    crime_dict = { key: [] for key in list(crimes[0].__dict__.keys())}

    crime_dict['lon'] = []
    crime_dict['lat'] = []
    crime_dict['crime_cat'] = []
    crime_dict['neighbourhood'] = []

    for crime in crimes:

      for key in list(crimes[0].__dict__.keys()):

        crime_dict[key] += [crime.__dict__[key]]

      # extract location lng and lat
      crime_dict['lon'] += [crime.__dict__['location'].__dict__['longitude']]

      crime_dict['lat'] += [crime.__dict__['location'].__dict__['latitude']]
      # extract crime category

      crime_dict['crime_cat'] += [crime.__dict__['category'].__dict__['name']]

      # add neighbourhood team name
      crime_dict['neighbourhood'] += [neighbourhood]
    
    pile_of_dataframes.append(pd.DataFrame(crime_dict))

  final_frame = pd.concat(pile_of_dataframes, axis=1)

  final_frame['police_force'] = selected_force.name

  return final_frame

In [ ]:
policeAPI_to_pandas(police_force='west-yorkshire', date="2019-06")

In [ ]:
crime_dict